In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [2]:
column_names = ['Page Popularity', 'Page Checkins', 'Page Talking About', 'Page Category', 
           'Derived_1', 'Derived_2', 'Derived_3', 'Derived_4', 'Derived_5', 
           'Derived_6', 'Derived_7', 'Derived_8', 'Derived_9', 'Derived_10',
           'Derived_11', 'Derived_12', 'Derived_13', 'Derived_14', 'Derived_15', 
           'Derived_16', 'Derived_17', 'Derived_14', 'Derived_19', 'Derived_20', 
           'Derived_21', 'Derived_22', 'Derived_15', 'Derived_24', 'Derived_25', 
           'CC1', 'CC2', 'CC3', 'CC4', 'CC5',
           'Base Time', 'Post Length', 'Post Share Count', 'Post Promotion Status', 'Local Hours',
           'Sunday Published', 'Monday Published', 'Tuesday Published', 'Wednesday Published', 'Thursday Published', 'Friday Published', 'Saturday Published', 
           'Sunday Base', 'Monday Base', 'Tuesday Base', 'Wednesday Base', 'Thursday Base', 'Friday Base', 'Saturday Base', 
           'target']

In [3]:
data = pd.read_csv('facebook_comment.csv', header=None, names=column_names)

D:\Program Files\Anaconda3\lib\site-packages\pandas\io\parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [4]:
display(data)

,Page Popularity,Page Checkins,Page Talking About,Page Category,Derived_1,Derived_2,Derived_3,Derived_4,Derived_5,Derived_6,...,Friday Published,Saturday Published,Sunday Base,Monday Base,Tuesday Base,Wednesday Base,Thursday Base,Friday Base,Saturday Base,target
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,0,1,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,0,0,0,0,0,1,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,1,0,0,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,1,0,0,0,0
5,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
6,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,0,1,0
7,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,0,1,0
8,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,0,0,0,0,0,1,0
9,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,1,0,0,0,0,0,0,0


## Нормирование

Заменим все NaN на 0.

In [5]:
if data.isna().any().sum() != 0:
    data.fillna(value=0, inplace=True)

Удалим все столбцы, в которых у всех элементов одинаковые значения.

In [6]:
drop_columns = []

In [7]:
print('Data Size: ', data.shape)

for column in data.columns:
    size = len(np.unique(data[column]))
    if size == 1:
        drop_columns += [column]

print('Drop Columns: ', drop_columns)

Data Size:  (40949, 54)
Drop Columns:  ['Post Promotion Status']


Преобразуем время

In [8]:
data.dtypes

Page Popularity            int64
Page Checkins              int64
Page Talking About         int64
Page Category              int64
Derived_1                float64
Derived_2                float64
Derived_3                float64
Derived_4                float64
Derived_5                float64
Derived_6                float64
Derived_7                float64
Derived_8                float64
Derived_9                float64
Derived_10               float64
Derived_11               float64
Derived_12               float64
Derived_13               float64
Derived_14               float64
Derived_15               float64
Derived_16               float64
Derived_17               float64
Derived_14.1             float64
Derived_19               float64
Derived_20               float64
Derived_21               float64
Derived_22               float64
Derived_15.1             float64
Derived_24               float64
Derived_25               float64
CC1                        int64
CC2       

In [9]:
print(np.unique(data['Local Hours']))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


In [10]:
def make_harmonic_features(value, period=24):
    value *= 2 * np.pi / period
    return np.cos(value), np.sin(value)

data['Harmonic Cos Local Hours'] = make_harmonic_features(data['Local Hours'], 24)[0]
data['Harmonic Sin Local Hours'] = make_harmonic_features(data['Local Hours'], 24)[1]

drop_columns += ['Local Hours']

In [11]:
data = data.drop(columns=drop_columns)

## Regression

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

In [13]:
def RMSE(y_test, y_pred):
    return np.sqrt(MSE(y_test, y_pred))

In [14]:
X = np.array(data.drop(columns=['target']))
y = np.array(data['target'])

In [15]:
model_lr = GradientBoostingRegressor()
model_coefs = []
model_stats = []

kf = KFold(n_splits=5)
kf.get_n_splits(X)
for train_index, test_index in kf.split(X):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    model_lr.fit(X_train, y_train)
    model_coefs += [model_lr.feature_importances_]
    y_pred = model_lr.predict(X_test)
    
    model_stats += [[np.average(y_test-y_pred), np.std(y_test-y_pred), MSE(y_test, y_pred), r2_score(y_test, y_pred)]]   

In [16]:
print('Feature Importance')
for n in range(len(model_coefs)):
    print(n, model_coefs[n])

Feature Importance
0 [6.65188180e-03 5.47246696e-03 1.15070510e-03 7.24962372e-04
 0.00000000e+00 2.14266753e-03 1.19055164e-03 6.55202340e-05
 1.05938227e-02 0.00000000e+00 2.41001627e-04 1.91299884e-02
 9.12218909e-03 1.04698923e-02 0.00000000e+00 6.03139807e-03
 8.33233608e-03 9.11671780e-03 7.23466101e-03 3.94055156e-04
 4.75389965e-04 1.52958755e-03 8.95847978e-04 1.16091570e-02
 1.04130872e-02 5.76818294e-04 2.47740263e-03 2.99116766e-03
 1.85067324e-03 1.93558622e-02 2.96118184e-01 2.40704945e-04
 2.20096942e-02 5.42978124e-02 3.30347729e-01 6.79009434e-03
 1.23745446e-01 0.00000000e+00 1.26279309e-04 4.83119920e-04
 1.30158124e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.43404722e-06 2.21188774e-03
 4.58712107e-05 4.08593872e-05 0.00000000e+00 0.00000000e+00
 2.78259462e-04]
1 [5.43284020e-03 7.27502932e-03 5.88983098e-03 4.39721102e-03
 4.34734097e-03 3.68797628e-04 8.49092165e-03 8.36423501e-03
 2.47789214e-03 1.14251299e-03 0.00000000e+00

In [17]:
print('Index', 'E', 'STD', 'MSE', 'R2')
for n in range(len(model_stats)):
    print(n, model_stats[n][0], model_stats[n][1], model_stats[n][2], model_stats[n][3])

Index E STD MSE R2
0 0.706123532144843 21.602293326851576 467.15768742198486 0.7030681632556688
1 0.5371314973814978 23.631740176507243 558.7476540154258 0.540241807688927
2 0.1582034091486883 20.553229132456444 422.4602560899226 0.7112798463793919
3 -1.0048211097246655 25.74261892578827 663.6920946209008 0.47932786896232726
4 -0.7681773100155244 17.36728886935775 302.2128190513404 0.6049678477744307
